## Streamlit App Entrypoint
The Streamlit interface has been refactored into regular Python modules.
Run `streamlit run streamlit_app.py` to launch the app with the refactored service layer.
This notebook delegates to the same `main()` function so the notebook experience stays consistent.


In [ ]:
import streamlit_app
streamlit_app.main()
